# Natural Language Processing - Text Preprocessing

## Libraries and settings

In [82]:
# Libraries
import os
import re
import string
import numpy as np
import pandas as pd
from pprint import pprint

import nltk

# Import only once
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.chunk import tree2conlltags
from nltk.chunk import conlltags2tree
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Current working directory
print('Current working directory:', os.getcwd())

Current working directory: /workspaces/data_analytics/Week_11


[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/vscode/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vscode/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Defining documents

In [83]:
# Defining documents (=sentenses)
d1 = 'The car is driven on the road.'
d2 = 'The truck is driven on the highway.'
d3 = 'The bicycle is driven on the bicycle path.'
d4 = 'The airplane is driven on the highway.'

corpus_01 = d1 + ' ' + d2 + ' ' + d3 + ' ' + d4
corpus_01

'The car is driven on the road. The truck is driven on the highway. The bicycle is driven on the bicycle path. The airplane is driven on the highway.'

## Text preprocessing
#### Steps:
- Text to lowercase
- Removing punctuations
- Tokenization
- Removal of stop words
- Lemmatization

### Text to lowercase

In [84]:
# Text to lowercase function
def text_lowercase(text):
    return text.lower()

# Text to lowercase
corpus_02 = text_lowercase(corpus_01)
corpus_02

'the car is driven on the road. the truck is driven on the highway. the bicycle is driven on the bicycle path. the airplane is driven on the highway.'

### Removing punctuation

In [85]:
# Remove punctuation function
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Remove punctuation
corpus_03 = remove_punctuation(corpus_02)
corpus_03

'the car is driven on the road the truck is driven on the highway the bicycle is driven on the bicycle path the airplane is driven on the highway'

### Tokenize text & removal of stopwords

In [86]:
# Show english stopwords
eng_stopwords = set(stopwords.words('english'))
print("List of english stopwords:")
print(eng_stopwords)

List of english stopwords:
{'too', 'where', 'only', "won't", 'of', 'their', 'under', 'further', 'into', 'so', 'more', 't', 'o', 'myself', 'itself', 'the', 'is', "didn't", 'do', 'ours', 'our', 'y', "mightn't", 'having', 'her', 'your', 'most', 'don', 'who', 'shan', "shan't", 'other', 'during', 'for', 'all', 'you', 'in', 'then', 'm', 'just', 'hadn', 'been', 'had', 'i', 'because', 'these', 'yourself', 'which', 'does', "couldn't", 'needn', 'a', "you'll", 'has', 'after', 'wouldn', 'how', 'out', 'ain', 'doesn', 'down', 'same', "you're", 'what', 'some', 'any', 'its', 'they', 'was', 've', "mustn't", "isn't", "you'd", 'each', 'd', 'wasn', 's', 'll', 'ourselves', 'with', 'below', "hadn't", 'my', 'as', 'here', 'an', 'own', "haven't", 'but', 'both', 'won', 'whom', 'me', 'haven', 'on', "needn't", 'few', 'those', "weren't", "you've", 'and', "don't", 'through', 'isn', "should've", 'have', 'mightn', 'themselves', 'mustn', "wasn't", 'or', 'theirs', 'nor', "wouldn't", 'while', 'between', 'when', 'not', '

In [87]:
# Function for tokenization and the removal of stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text
 
# Remove stopwords
corpus_04 = remove_stopwords(corpus_03)
print(corpus_04, end="")

['car', 'driven', 'road', 'truck', 'driven', 'highway', 'bicycle', 'driven', 'bicycle', 'path', 'airplane', 'driven', 'highway']

### Lemmatization

In [88]:
# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize string function
def lemmatize_word(text):
    word_tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    return lemmas

# Lemmatize
lem = []
for i in corpus_04:
    lem.append(lemmatize_word(i))

# Nested list to list
corpus_05 = [' '.join([str(x) for x in lst]) for lst in lem]

print('Before lemmatization:')
print(corpus_04, '\n')

print('After lemmatization:')
print(corpus_05, end="")

Before lemmatization:
['car', 'driven', 'road', 'truck', 'driven', 'highway', 'bicycle', 'driven', 'bicycle', 'path', 'airplane', 'driven', 'highway'] 

After lemmatization:
['car', 'drive', 'road', 'truck', 'drive', 'highway', 'bicycle', 'drive', 'bicycle', 'path', 'airplane', 'drive', 'highway']

## Redefine the text corpus (pre-processed)

In [89]:
# We will use the lemmatized words above to re-define our corpus 
corpus = ['car drive road', 
          'truck drive highway', 
          'bicycle drive bicycle path',
          'airplane drive highway']

## Document-term matrix with ngram_range=(1,1)

In [90]:
# Vectorizer with ngram_range=(1,1)
vectorizer = CountVectorizer(min_df=0.0, ngram_range=(1,1))

# Transform 
count = vectorizer.fit_transform(corpus)
 
# Create dataframe
df_count = pd.DataFrame(count.toarray(),
                        columns=vectorizer.get_feature_names_out())

print('Document-term matrix')
print(df_count)

Document-term matrix
   airplane  bicycle  car  drive  highway  path  road  truck
0         0        0    1      1        0     0     1      0
1         0        0    0      1        1     0     0      1
2         0        2    0      1        0     1     0      0
3         1        0    0      1        1     0     0      0


## Document-term matrix with ngram_range=(2,2)

In [91]:
# Vectorizer with with ngram_range=(2,2)
vectorizer = CountVectorizer(min_df=0.0, ngram_range=(2,2))

# Transform 
count = vectorizer.fit_transform(corpus)
 
# Create dataframe
df_count = pd.DataFrame(count.toarray(),
                        columns=vectorizer.get_feature_names_out())

print('Document-term matrix')
print(df_count)

Document-term matrix
   airplane drive  bicycle drive  bicycle path  car drive  drive bicycle  \
0               0              0             0          1              0   
1               0              0             0          0              0   
2               0              1             1          0              1   
3               1              0             0          0              0   

   drive highway  drive road  truck drive  
0              0           1            0  
1              1           0            1  
2              0           0            0  
3              1           0            0  


## Term frequency-inverse document frequency (TF-IDF)
- For details see: https://www.learndatasci.com/glossary/tf-idf-term-frequency-inverse-document-frequency

### Term Frequency (TF)

In [92]:
# Compute Term Frequency (TF)
words_set = set()
for doc in corpus:
    words = doc.split(' ')
    words_set = words_set.union(set(words))
    
print('Number of words in the corpus:',len(words_set), '\n')
print('The words in the corpus: \n', words_set)

# Number of documents in the corpus
n_docs = len(corpus)

# Number of unique words in the corpus 
n_words_set = len(words_set)

df_tf = pd.DataFrame(np.zeros((n_docs, n_words_set)), 
                     columns=list(words_set))

print("\nTerm Frequency (TF):")
for i in range(n_docs):
    # Words in the document
    words = corpus[i].split(' ')
    for w in words:
        df_tf[w][i] = df_tf[w][i] + (1 / len(words))
        
print(df_tf.round(4))

Number of words in the corpus: 8 

The words in the corpus: 
 {'drive', 'truck', 'car', 'bicycle', 'road', 'airplane', 'highway', 'path'}

Term Frequency (TF):
    drive   truck     car  bicycle    road  airplane  highway  path
0  0.3333  0.0000  0.3333      0.0  0.3333    0.0000   0.0000  0.00
1  0.3333  0.3333  0.0000      0.0  0.0000    0.0000   0.3333  0.00
2  0.2500  0.0000  0.0000      0.5  0.0000    0.0000   0.0000  0.25
3  0.3333  0.0000  0.0000      0.0  0.0000    0.3333   0.3333  0.00


### Inverse Document Frequency (IDF)

In [93]:
# Computing Inverse Document Frequency (IDF)
print("\nInverse Document Frequency (IDF):")

idf = {}

for w in words_set:
    
    # k = number of documents that contain this word
    k = 0
    
    for i in range(n_docs):
        if w in corpus[i].split():
            k += 1
            
    idf[w] =  np.log10(n_docs / k).round(4)
    
    print(f'{w:>15}: {idf[w]:>10}')


Inverse Document Frequency (IDF):
          drive:        0.0
          truck:     0.6021
            car:     0.6021
        bicycle:     0.6021
           road:     0.6021
       airplane:     0.6021
        highway:      0.301
           path:     0.6021


### Term Frequency - Inverse Document Frequency (TF-IDF)

In [94]:
# Computing TF-IDF
df_tf_idf = df_tf.copy()

for w in words_set:
    for i in range(n_docs):
        df_tf_idf[w][i] = df_tf[w][i] * idf[w]

print('\nTF-IDF:')
print(df_tf_idf.round(4))


TF-IDF:
   drive   truck     car  bicycle    road  airplane  highway    path
0    0.0  0.0000  0.2007    0.000  0.2007    0.0000   0.0000  0.0000
1    0.0  0.2007  0.0000    0.000  0.0000    0.0000   0.1003  0.0000
2    0.0  0.0000  0.0000    0.301  0.0000    0.0000   0.0000  0.1505
3    0.0  0.0000  0.0000    0.000  0.0000    0.2007   0.1003  0.0000


## Part-of-Speach (POS) tagging
For meaning of POS-tags see: https://pythonexamples.org/nltk-pos-tagging

In [95]:
text = '''The cat sat by the window, watching birds fly by, while the gentle breeze moved the curtains softly around.'''

def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

sent = preprocess(text)
pattern = 'NP: {<DT>?<JJ>*<NN>}'

cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)

iob_tagged = tree2conlltags(cs)

# Print the POS-tags
pprint(iob_tagged)

[('The', 'DT', 'B-NP'),
 ('cat', 'NN', 'I-NP'),
 ('sat', 'VBN', 'O'),
 ('by', 'IN', 'O'),
 ('the', 'DT', 'B-NP'),
 ('window', 'NN', 'I-NP'),
 (',', ',', 'O'),
 ('watching', 'VBG', 'O'),
 ('birds', 'NNS', 'O'),
 ('fly', 'VBN', 'O'),
 ('by', 'IN', 'O'),
 (',', ',', 'O'),
 ('while', 'IN', 'O'),
 ('the', 'DT', 'B-NP'),
 ('gentle', 'JJ', 'I-NP'),
 ('breeze', 'NN', 'I-NP'),
 ('moved', 'VBD', 'O'),
 ('the', 'DT', 'O'),
 ('curtains', 'NNS', 'O'),
 ('softly', 'RB', 'O'),
 ('around', 'IN', 'O'),
 ('.', '.', 'O')]


**EXPLAINATION:**
- **DT**: Determiner (*the, a, an*)  
- **NN**: Singular Noun (*cat, window*)  
- **VBN**: Past Participle (*sat, fly*)  
- **IN**: Preposition/Subordinating Conjunction (*by, while*)  
- **,**: Comma  
- **VBG**: Present Participle (*watching*)  
- **NNS**: Plural Noun (*birds, curtains*)  
- **JJ**: Adjective (*gentle*)  
- **VBD**: Past Tense (*moved*)  
- **RB**: Adverb (*softly*)  
- **.**: Period

<BR>

- ('The', 'DT', 'B-NP')    → Anfang einer Nominalphrase
- ('cat', 'NN', 'I-NP')    → Innerhalb der gleichen Nominalphrase
- ('sat', 'VBN', 'O')      → Gehört zu keiner Phrase

*Eine Nominalphrase konzentriert sich auf ein Nomen, oft begleitet von beschreibenden Wörtern, die mehr Details liefern.*


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [96]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.5.0-1025-azure
Datetime: 2024-11-29 08:09:21
Python Version: 3.11.10
-----------------------------------
